In [1]:
model_name = "ai4bharat/indic-bert"

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import load_dataset, Dataset, ClassLabel
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np
import pandas as pd

torch.cuda.empty_cache()

In [3]:
df = pd.read_excel('Labelled.xlsx')

In [4]:
df = df[df['label'].notnull()]

In [5]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)


In [6]:
data_train = Dataset.from_pandas(train_data, preserve_index=False)
data_test = Dataset.from_pandas(test_data, preserve_index=False)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

labels = ClassLabel(num_classes=4, names=['Offensive-Ind', 'Not-Offensive', 'Offensive-Group','Offensive-Untargetted'], names_file=None, id=None)

def tokenize_function(examples):
    tokens =  tokenizer(examples["text"], padding=True, truncation=True, max_length=128)
    tokens['label'] = labels.str2int(examples['label'])
    return tokens

tokenized_train = data_train.map(tokenize_function, batched=True)
tokenized_test = data_test.map(tokenize_function, batched=True)

tokenized_train.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
tokenized_test.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Parameter 'function'=<function tokenize_function at 0x7fe5b5684a60> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [8]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
id2label = {idx:labels.int2str(idx) for idx in range(4)}
label2id = {v:k for k,v in id2label.items()}

In [10]:
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        #Feed inputs to model and extract logits
        outputs = model(**inputs)
        logits = outputs.get("logits")
        #Extract labels
        labels = inputs.get("labels")
        # Define loss function with class weights
        loss_func = nn.CrossEntropyLoss()
        # Compute loss
        loss = loss_func(logits, labels)
        return (loss, outputs) if return_outputs else loss 

In [11]:
model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                             num_labels=4,
                                                             id2label=id2label,
                                                             label2id=label2id)

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForSequenceClassification: ['predictions.dense.weight', 'sop_classifier.classifier.bias', 'predictions.LayerNorm.bias', 'predictions.decoder.weight', 'sop_classifier.classifier.weight', 'predictions.dense.bias', 'predictions.LayerNorm.weight', 'predictions.bias', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indi

In [12]:
def compute_metrics(pred):

    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    accuracy = accuracy_score(y_true=labels, y_pred=preds)
    recall = recall_score(y_true=labels, y_pred=preds, average='weighted')
    precision = precision_score(y_true=labels, y_pred=preds, average='weighted')
    f1 = f1_score(y_true=labels, y_pred=preds, average='weighted')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [13]:
batch_size = 16
epochs = 30
leaning_rate = 2e-5


# Log the training loss at each epoch
logging_steps = len(tokenized_train) // batch_size

training_args = TrainingArguments(output_dir='Indic-BERT',
                                    num_train_epochs=epochs,
                                    learning_rate=leaning_rate,
                                    per_device_train_batch_size=batch_size,
                                    per_device_eval_batch_size=batch_size,
                                    weight_decay=0.01,
                                    evaluation_strategy="steps",
                                    logging_steps=logging_steps,
                                    fp16=True,
                                    eval_steps=250,
                                    load_best_model_at_end=True)

In [14]:
trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

Using cuda_amp half precision backend


In [15]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `AlbertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 4013
  Num Epochs = 30
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3780


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
250,0.881000,0.762331,0.738048,0.613378,0.738048,0.668380
500,0.682600,0.755689,0.749004,0.635251,0.749004,0.687436
750,0.497700,0.768505,0.754980,0.711879,0.754980,0.727201
1000,0.334500,0.965746,0.701195,0.728816,0.701195,0.712141
1250,0.235400,0.946331,0.743028,0.734224,0.743028,0.731832
1500,0.162700,1.059826,0.753984,0.740474,0.753984,0.742917
1750,0.112700,1.192405,0.725100,0.739111,0.725100,0.728623


The following columns in the evaluation set don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `AlbertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1004
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `AlbertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1004
  Batch size = 32
Saving model checkpoint to Indic-BERT/checkpoint-500
Configuration saved in Indic-BERT/checkpoint-500/config.json
Model weights saved in Indic-BERT/checkpoint-500/pytorch_model.bin
tokenizer config file saved in Indic-BERT/checkpoint-500/tokenizer_config.json
Special tokens file saved in Indic-BERT/checkpoint-500/special_tokens_map.

TrainOutput(global_step=1750, training_loss=0.41524141911097934, metrics={'train_runtime': 256.6, 'train_samples_per_second': 469.174, 'train_steps_per_second': 14.731, 'total_flos': 299325432371352.0, 'train_loss': 0.41524141911097934, 'epoch': 13.89})

In [2]:
from transformers import TextClassificationPipeline, AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import  ClassLabel


df = pd.read_excel('Labelled.xlsx')
df = df[df['label'].notnull()]
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

model_name = "Indic-BERT/checkpoint-500"
tokenizer = AutoTokenizer.from_pretrained(model_name)


labels = ClassLabel(num_classes=4, names=['Offensive-Ind', 'Not-Offensive', 'Offensive-Group','Offensive-Untargetted'], names_file=None, id=None)
id2label = {idx:labels.int2str(idx) for idx in range(4)}
label2id = {v:k for k,v in id2label.items()}

model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=4,id2label=id2label,label2id=label2id)


pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=False)
y_pred = pipe(test_data['text'].tolist())
y_pred = [y_pred[i]["label"] for i in range(len(y_pred))]
y_true = test_data['label'].tolist()
print(classification_report(y_true, y_pred))

/home/sudheesh/anaconda3/envs/rubrix/lib/python3.8/site-packages/transformers/pipelines/text_classification.py:89: UserWarning: `return_all_scores` is now deprecated, use `top_k=1` if you want similar functionnality
  warnings.warn(


                       precision    recall  f1-score   support

        Not-Offensive       0.00      0.00      0.00        68
      Offensive-Group       0.49      0.59      0.53       125
        Offensive-Ind       0.80      0.94      0.86       725
Offensive-Untargetted       0.00      0.00      0.00        86

             accuracy                           0.75      1004
            macro avg       0.32      0.38      0.35      1004
         weighted avg       0.64      0.75      0.69      1004



/home/sudheesh/anaconda3/envs/rubrix/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sudheesh/anaconda3/envs/rubrix/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sudheesh/anaconda3/envs/rubrix/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le